# Week 2 Exercise: API Docs Q&A

A small **API documentation Q&A assistant**. It uses a lightweight retrieval step (a tool) to pull relevant doc sections, then answers your question with citations.

**Goal:** demonstrate tool usage + retrieval + structured responses.


In [1]:
# Imports
import os
import re
import json
from dotenv import load_dotenv
from openai import OpenAI


In [2]:
# Load environment variables (.env)
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if not api_key:
    print('No API key found. Please add OPENAI_API_KEY to your .env file.')
elif api_key.strip() != api_key:
    print('API key has leading/trailing whitespace. Please remove it.')
else:
    print('API key looks good!')

openai = OpenAI(
    api_key=api_key,
    base_url='https://openrouter.ai/api/v1',
)


API key looks good!


In [3]:
# Model selection
MODEL = 'gpt-4.1-mini'


In [12]:
# Sample API docs (replace with your own)
API_DOCS = '''
# Acme Payments API

## Authentication
Use a bearer token in the Authorization header. Tokens expire after 24 hours.

## Create Charge (POST /v1/charges)
Required fields: amount (integer, cents), currency (string), source (string token).
Optional: description, metadata.
Returns: charge_id, status, created_at.

## Refund Charge (POST /v1/charges/{charge_id}/refunds)
Required fields: amount (integer, cents).
Optional: reason (string).
Returns: refund_id, status.

## List Charges (GET /v1/charges)
Query params: status, limit, starting_after.
Returns a paginated list of charges.

## Webhooks
We send events for charge.succeeded, charge.failed, refund.created.
Retry policy: 3 attempts over 24 hours.
'''


In [13]:
# Simple doc chunking by headings
def chunk_docs(text: str):
    chunks = []
    current = []
    for line in text.splitlines():
        if line.startswith('#'):  # new section
            if current:
                chunks.append('\n'.join(current).strip())
                current = []
        current.append(line)
    if current:
        chunks.append('\n'.join(current).strip())
    return [c for c in chunks if c]

DOC_CHUNKS = chunk_docs(API_DOCS)


In [14]:
# Retrieval tool: return top-k sections by keyword overlap
def search_docs(query: str, k: int = 3):
    q = re.findall(r'\w+', query.lower())
    if not q:
        return []
    scores = []
    for chunk in DOC_CHUNKS:
        text = chunk.lower()
        score = sum(text.count(word) for word in q)
        scores.append((score, chunk))
    scores.sort(key=lambda x: x[0], reverse=True)
    return [c for s, c in scores if s > 0][:k]


In [15]:
# Tool definition for function calling
TOOLS = [
    {
        'type': 'function',
        'function': {
            'name': 'search_docs',
            'description': 'Search the API docs and return relevant sections.',
            'parameters': {
                'type': 'object',
                'properties': {
                    'query': {'type': 'string'},
                    'k': {'type': 'integer', 'default': 3},
                },
                'required': ['query']
            }
        }
    }
]


In [16]:
SYSTEM_PROMPT = '''
You are an API documentation assistant.
When needed, call the search_docs tool to retrieve relevant sections.
You MUST respond using these exact headings (with ##):
## Short Answer
## Details
## Citations
Under Citations, use bullet points and quote the section titles used.
If the docs don't mention it, say so clearly.
If you fail to use the exact headings, the answer is invalid.
Example format:
## Short Answer
...
## Details
...
## Citations
- "Section Title"
'''


In [17]:
def ask(question: str):
    messages = [
        {'role': 'system', 'content': SYSTEM_PROMPT},
        {'role': 'user', 'content': question},
    ]

    response = openai.chat.completions.create(
        model=MODEL,
        messages=messages,
        tools=TOOLS,
        tool_choice='auto',
        max_tokens=800,
    )

    msg = response.choices[0].message
    if msg.tool_calls:
        # Handle tool calls
        tool_outputs = []
        for call in msg.tool_calls:
            if call.function.name == 'search_docs':
                args = json.loads(call.function.arguments) if hasattr(call.function, 'arguments') else {}
                query = args.get('query', question)
                k = args.get('k', 3)
                results = search_docs(query, k=k)
                tool_outputs.append({
                    'tool_call_id': call.id,
                    'role': 'tool',
                    'name': 'search_docs',
                    'content': '\n\n'.join(results) if results else 'No relevant sections found.'
                })

        # Send tool outputs back to the model
        response = openai.chat.completions.create(
            model=MODEL,
            messages=messages + [msg] + tool_outputs,
            max_tokens=800,
        )
        return response.choices[0].message.content

    return msg.content


In [ ]:
# Example questions (run one at a time)
print(ask('How do I refund a charge?'))
print(ask('What webhook events exist, and how often are retries?'))


## Short Answer
You refund a charge by making a POST request to the endpoint `/v1/charges/{charge_id}/refunds` with the required amount to refund and optional reason.

## Details
To refund a charge, use the endpoint `POST /v1/charges/{charge_id}/refunds`. You need to specify the amount to refund in cents as a required field. Optionally, you can provide a reason for the refund. The response will include the refund ID and status of the refund.

## Citations
- "Refund Charge (POST /v1/charges/{charge_id}/refunds)"
